In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt                               
import os 
import matplotlib.colors as mcolors
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import ipywidgets as widgets

In [2]:
cmap = mcolors.LinearSegmentedColormap.from_list("", ["white","red","darkred"])

In [3]:
def plotCluster(index, timeSlice):
    label = truth.iloc[index]
    cluster = recon2D.iloc[index].to_numpy().reshape(13,21)
    clusterSlice = recon3D.iloc[index].to_numpy().reshape(20,13,21)[timeSlice]
    
    fig, ax = plt.subplots(1,2, figsize=(12,5),dpi=200)

    # Plot charge collected in each pixel
    datamin = cluster.min()
    datamax = cluster.max()
    #vmax = max(abs(datamin),abs(datamax))
    #vmin = -vmax
    im = ax[1].imshow(cluster, vmin=datamin, vmax=datamax, cmap=cmap,interpolation='nearest')
    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes('right', size='4%', pad=0.05)
    fig.colorbar(im, cax=cax, location='right',label='Number of eh pairs')
    ax[1].set_title("Overall charge collected")

    im = ax[0].imshow(clusterSlice, vmin=datamin, vmax=datamax, cmap=cmap, interpolation='nearest')
    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes('right', size='4%', pad=0.05)
    fig.colorbar(im, cax=cax, location='right',label='Number of eh pairs')
    ax[0].set_title(f"Charge collected by {round(timeSlice*.2+.2, 2)} ns")

    # Draw grid on both
    for axis in ax:
        axis.set_xlim(-0.5,20.5)
        axis.set_ylim(-0.5,12.5)
        axis.xaxis.set_major_locator(ticker.MultipleLocator(1))
        axis.yaxis.set_major_locator(ticker.MultipleLocator(1))
        axis.xaxis.set_minor_locator(ticker.MultipleLocator(0.5))
        axis.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
        plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
        plt.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
        axis.grid(which="minor", color="grey", linestyle='-', linewidth=0.5,snap=False)
    
    # Adjust truth values for coordinate system (the origin is in the center of the ROI)
    xentry = label['x-entry']/50 + 21/2
    yentry = label['y-entry']/12.5 + 13/2
    xmid = label['x-midplane']/50 + 21/2
    ymid = label['y-midplane']/12.5 + 13/2
    
    ax[1].plot(xentry,yentry, 'b.',)
    ax[1].plot(xmid,ymid, 'g.',)
    ax[1].arrow(xentry, yentry, label['n_x']/50*100, label['n_y']/12.5*100, color='b',length_includes_head=True, head_width=.3, head_length=.3)
    
    plt.tight_layout(pad=3.5)
    fig.canvas.draw()

In [5]:
datadir = '/home/elizahoward/cmspix28-mc-sim/MuC_Output'
tag = "sig_2"
truth = pd.read_parquet(f"{datadir}/labels{tag}.parquet")
recon2D = pd.read_parquet(f"{datadir}/recon2D{tag}.parquet")
recon3D = pd.read_parquet(f"{datadir}/recon3D{tag}.parquet")
widgets.interact(plotCluster, index=(0,50,1), timeSlice=(0,19,1))

interactive(children=(IntSlider(value=25, description='index', max=50), IntSlider(value=9, description='timeSl…

<function __main__.plotCluster(index, timeSlice)>